In [ ]:
import numpy as np
import pandas as pd
import sqlalchemy as sa
import pudl

import tarfile

import splink

from pudl import logging_helpers
logger = pudl.logging_helpers.get_logger(__name__)

# CorpsWatch SEC EX-21 download
I downloaded the CSV's from this file:
https://archive.org/download/corpwatch_api_data_dumps/corpwatch_api_tables_csv.tar.gz

Unzipped the tarball and put it in a dir in *this* directory.

# SEC EX-21 Extract
I downloaded the CSv's from this file:
https://archive.org/download/corpwatch_api_data_dumps/corpwatch_api_tables_csv.tar.gz

And put it in a dir in *this* directory.

In [ ]:
directory = "corpwatch_api_tables_csv"
# tables to grab
tables_sec21 = ["companies", "company_relations", "company_info", "sic_codes", "company_locations"]

for table in tables_sec21:
    logger.info(f"{table}: extracting CSV")
    file = tarfile.open(f"../{directory}/corpwatch_api_tables_csv.tar.gz")
    file.extract(f"corpwatch_api_tables_csv/{table}.csv", f"../{directory}")
    file.close()
    vars()[table] = pd.read_csv(f"../{directory}/corpwatch_api_tables_csv/{table}.csv", sep='\t').convert_dtypes()

In [ ]:
# there are nulls for cw_id, but otherwise it is unique
companies[companies.duplicated(subset=["cw_id"], keep=False)]

In [ ]:
# doesn't seem like null cw_id's have other company records
companies[companies.company_name.str.contains("Dentrix")]

In [ ]:
company_relations.head(3)

In [ ]:
company_info.head(3)

In [ ]:
test = company_info.merge(sic_codes, on=["sic_code", "industry_name", "sic_sector"], how="left", indicator=True)

In [ ]:
# there are some sic codes in the company info table that aren't in the sic_codes table
test[(test._merge=="left_only") & ~(test.sic_code.isnull())].sic_code.unique()

In [ ]:
sic_codes[sic_codes.sic_code==6120]

In [ ]:
company_locations.location_id.is_unique

In [ ]:
# min_year and max_year represent how long an address is applicable for, there might be a change in address after max_year
company_locations[company_locations.street_1.str.contains("PITMAN ROAD")]

In [ ]:
# it's not entirely clear what this date column represents in the company locations, the date that the record was entered?
loc_test = (
        company_locations.dropna(subset=["cw_id"])
        .assign(loc_year=lambda x: pd.to_datetime(company_locations.date).dt.year).convert_dtypes()
    )

In [ ]:
loc_test[loc_test.min_year != loc_test.loc_year].head(3)

In [ ]:
# there will be a new location id for the same company when the location changes
# there will also be a different record for each location type
loc_test[loc_test.cw_id == 1]

In [ ]:
loc_test[loc_test.location_id == 1]

In [ ]:
# try merging on best_location_id
loc_id_only = company_info.merge(loc_test, how="left", left_on="best_location_id", right_on="location_id", indicator=True)

In [ ]:
# there's one location id that isn't in the locations df
loc_id_only[loc_id_only._merge == "left_only"].best_location_id.unique()

In [ ]:
# seems like best_location_id locations correspond to the correct companies
loc_id_only[loc_id_only.cw_id_x != loc_id_only.cw_id_y]

In [ ]:
# so potentially some of these best_location_id's are wrong, are there better location_id's to use?
loc_id_only[(loc_id_only.year > loc_id_only.max_year_y) | (loc_id_only.year < loc_id_only.min_year_y)].head(3)

In [ ]:
# a very small percentage have years that don't fall within the correct range
len(loc_id_only[(loc_id_only.year > loc_id_only.max_year_y) | (loc_id_only.year < loc_id_only.min_year_y)])/len(loc_id_only)

In [ ]:
# for example this company changed locations in 2015, so the location id in the company info should have been updated
company_locations[company_locations.cw_id == 635924]

Summary:
- `best_location_id` is mostly reliable, .02% of records have a `best_location_id` where the the location changed and it's using the previous location
- this could be fixed by taking these bad records and getting the record for that `cw_id` and `type` from the `company_location` dataframe that has a `min_year` and `max_year` that contains the `year` for the record
- for now it seems fine to use `best_location_id` instead of using `cw_id`, `year`, and `type` as the join key
- this `max_year`, `min_year`, `year` issue doesn't seem to be an issue for electricity sector companies

In [ ]:
elec = company_info[company_info.sic_sector.isin([4900])]

In [ ]:
elec_loc = elec.merge(loc_test, how="left", left_on="best_location_id", right_on="location_id")

In [ ]:
len(elec_loc[(elec_loc.year > elec_loc.max_year_y) | (elec_loc.year < elec_loc.min_year_y)])/len(elec_loc)

# Input Prep

In [ ]:
idx_eia = ["utility_id_eia", "report_date"]
# when you use best_location as the merge key you no longer need type in the key
idx_sec = ["cw_id", "loc_year", "type"]
shared_cols = ["year", "utility_name_eia", "city", "street_address", "address_2", "zip_code"]

## `pudl_out` setup

In [ ]:
pudl_settings = pudl.workspace.setup.get_defaults()
pudl_engine = sa.create_engine(pudl_settings['pudl_db'])

pudl_out = pudl.output.pudltabl.PudlTabl(pudl_engine=pudl_engine)
utils_eia860 = pudl_out.utils_eia860().assign(year=lambda x: x.report_date.dt.year)

## SEC setup

In [ ]:
def prep_companies_sec(company_info, company_locations):
    # either grab both sectors that have 
    sic_sector_electric = (
        # company_info.loc[company_info.sector_name.str.lower().str.contains("electric"), "sic_sector"].unique()
        [4900]
    )
    # maybe we don't need this comapny year anymore
    company_locations = (
        company_locations.dropna(subset=["cw_id"])
        .assign(loc_year=lambda x: pd.to_datetime(company_locations.date).dt.year).convert_dtypes()
    )

    dupe_address_ratio = (
        len(company_locations[company_locations.duplicated(subset=idx_sec, keep=False)])
        /len(company_locations)
    )
    if dupe_address_ratio > .0001:
        raise AssertionError(
            f"{dupe_address_ratio:.3%} of addresses are duplicates based on {idx_sec}. "
            "More than expected (.01%)."
        )
    companies_sec = (
        company_info
        [company_info.sic_sector.isin(sic_sector_electric)]
        .merge(
            company_locations,
            left_on="best_location_id",
            right_on="location_id",
            validate="m:1",
            how="left"
        )
        .rename(columns={
            "company_name": "utility_name_eia",
            "city": "city",
            "street_1": "street_address",
            "street_2": "address_2",
            "postal_code": "zip_code"
        })
    )
    return companies_sec

In [ ]:
companies_sec = prep_companies_sec(company_info, company_locations)

In [ ]:
companies_sec.head(3)

In [ ]:
utils_eia860[shared_cols]

In [ ]:
companies_sec[shared_cols]

# Cleaning

In [ ]:
len(companies_sec[companies_sec.street_address.isnull()])/len(companies_sec)

In [ ]:
utils_eia860.loc[(utils_eia860.street_address.isnull()) & ~(utils_eia860.address_2.isnull()), "street_address"] = utils_eia860.address_2

In [ ]:
len(utils_eia860[utils_eia860.street_address.isnull()])/len(utils_eia860)

In [ ]:
null_util = utils_eia860[utils_eia860.street_address.isnull()][shared_cols]

In [ ]:
null_util.city.isnull().value_counts()

Other cleaning steps that should be done:
- replace special characters from strings #, ', .
- replace abbreviations with full names - st. -> street

In [ ]:
def clean_df(df):
    df = df[shared_cols]
    types = {"utility_name_eia": str, "city": str, "street_address": str, "address_2": str, "zip_code": str, "year": int}
    df = df.astype(types)
    str_cols = ["utility_name_eia", "city", "street_address", "address_2"]
    for col in str_cols:
        df[col] = df[col].str.strip().str.lower()
    # fix nans with n number of x's
    df = df.replace(["-", ".", "<na>", "<NA>", "nan", "NaN", "x", "xxx", "xxxxxxx"], np.nan)
    df.loc[~(df.zip_code.isnull()), "zip_code"] = df["zip_code"].str.zfill(5)
    # replace null addresses with address_2 if it's not null
    # is this a good idea?
    df.loc[(df.street_address.isnull()) & ~(df.address_2.isnull()), "street_address"] = df.address_2
    df = df.drop_duplicates()
    df = df.reset_index(drop=True)
    # add unique_id column for splink
    df["unique_id"] = df.index
    return df

In [ ]:
sec_df = clean_df(companies_sec)
eia_df = clean_df(utils_eia860)

# Splink Exploratory Analysis

In [ ]:
from splink.duckdb.duckdb_linker import DuckDBLinker

In [ ]:
# build up settings dict more later
settings_dict = {"link_type": "link_only"}
linker = DuckDBLinker([sec_df, eia_df], settings_dict=settings_dict)

In [ ]:
linker.missingness_chart()

Strong skew in city column.

Name and address are likely the best linking columns, as is expected. Low value counts for each of the values.

In [ ]:
cols = ["utility_name_eia", "city", "street_address", "address_2", "zip_code"]
linker.profile_columns(cols)

### Look at blocking rules

In [ ]:
# this creates too many records, can you make it so that it's an "and" with all the other rules as "ors"
blocking_rule_1 = "l.year = r.year"
count = linker.count_num_comparisons_from_blocking_rule(blocking_rule_1)
print(f"Number of comparisons generated by '{blocking_rule_1}': {count:,.0f}")

In [ ]:
blocking_rule_2 = "l.zip_code = r.zip_code and l.year = r.year"
count = linker.count_num_comparisons_from_blocking_rule(blocking_rule_2)
print(f"Number of comparisons generated by '{blocking_rule_2}': {count:,.0f}")

In [ ]:
blocking_rule_3 = "l.city = r.city and l.year = r.year"
count = linker.count_num_comparisons_from_blocking_rule(blocking_rule_3)
print(f"Number of comparisons generated by '{blocking_rule_3}': {count:,.0f}")

In [ ]:
blocking_rule_4 = "substr(l.utility_name_eia, 1, 4) = substr(r.utility_name_eia, 1, 4) and l.year = r.year"
count = linker.count_num_comparisons_from_blocking_rule(blocking_rule_4)
print(f"Number of comparisons generated by '{blocking_rule_4}': {count:,.0f}")

In [ ]:
blocking_rule_5 = "substr(l.street_address, 1, 5) = substr(r.street_address, 1, 5) and l.year = r.year"
count = linker.count_num_comparisons_from_blocking_rule(blocking_rule_5)
print(f"Number of comparisons generated by '{blocking_rule_5}': {count:,.0f}")

In [ ]:
blocking_rule_6 = "l.address_2 = r.address_2 and l.year = r.year"
count = linker.count_num_comparisons_from_blocking_rule(blocking_rule_6)
print(f"Number of comparisons generated by '{blocking_rule_6}': {count:,.0f}")

In [ ]:
blocking_rule_6 = "substr(l.address_2, 1, 9) = substr(r.address_2, 1, 9) and l.year = r.year"

In [ ]:
blocking_rules = [blocking_rule_2, blocking_rule_3, blocking_rule_4, blocking_rule_5, blocking_rule_6]
linker.cumulative_num_comparisons_from_blocking_rules_chart(blocking_rules)

Ideas:
Do you want to dedupe the records on year and then link?  
For now just run with one year of data?

In [ ]:
# blocking rules if we just have one year of data
br_1 = "l.zip_code = r.zip_code"
br_2 = "l.city = r.city"
br_3 = "substr(l.utility_name_eia, 1, 4) = substr(r.utility_name_eia, 1, 4)"
br_4 = "substr(l.street_address, 1, 5) = substr(r.street_address, 1, 5)"
br_5 = "substr(l.address_2, 1, 9) = substr(r.address_2, 1, 9)"
blocking_rules = [br_1, br_2, br_3, br_4, br_5]

In [ ]:
sec_df = clean_df(sec_df)
eia_df = clean_df(eia_df)

In [ ]:
sec_df_2005 = sec_df[sec_df.year == 2005]
eia_df_2005 = eia_df[eia_df.year == 2005]

TODO: Fix this later

In [ ]:
eia_df_2005.iloc[7331].zip_code

In [ ]:
eia_df_2005

In [ ]:
sec_df

# Link datasets

In [ ]:
import splink.duckdb.duckdb_comparison_library as cl

utility_name_comparison = cl.levenshtein_at_thresholds("utility_name_eia", [2, 5])
print(utility_name_comparison.human_readable_description)

In [ ]:
settings = {
    "link_type": "link_only",
    "comparisons": [
        utility_name_comparison,
        cl.levenshtein_at_thresholds("city", 3, term_frequency_adjustments=True),
        cl.levenshtein_at_thresholds("street_address", [2, 5]),
        cl.levenshtein_at_thresholds("address_2", 2),
        cl.exact_match("zip_code"),
    ],
    "blocking_rules_to_generate_predictions": blocking_rules,
    "retain_matching_columns": True,
    "retain_intermediate_calculation_columns": True,
}

In [ ]:
linker = DuckDBLinker([sec_df_2005, eia_df_2005], settings_dict=settings)

Get the probability two random records match

In [ ]:
deterministic_rules = [
    "l.utility_name_eia = r.utility_name_eia and levenshtein(l.street_address, r.street_address) <= 2",
    "l.utility_name_eia = r.utility_name_eia and levenshtein(r.city, l.city) <= 2",
    "l.city = r.city and levenshtein(l.utility_name_eia, r.utility_name_eia) <= 2",
    "l.zip_code = r.zip_code and levenshtein(l.utility_name_eia, r.utility_name_eia) <= 2",
    "l.street_address = r.street_address"
]

linker.estimate_probability_two_random_records_match(deterministic_rules, recall=0.7)


Estimate the u parameter

In [ ]:
linker.estimate_u_using_random_sampling(target_rows=1e6)

Estimate the m parameter

In [ ]:
training_blocking_rule = "l.utility_name_eia = r.utility_name_eia and l.city = r.city"
training_session_fname_sname = linker.estimate_parameters_using_expectation_maximisation(training_blocking_rule)

In [ ]:
training_blocking_rule = "l.street_address = r.street_address"
training_session_fname_sname = linker.estimate_parameters_using_expectation_maximisation(training_blocking_rule)

Seems like another the city comparison could be changed to have levels 1, 3, all other. Might make the all other less negative.

City and zip code are probably very dependent, maybe don't include zip code?

In [ ]:
linker.match_weights_chart()

Need more emphasis on utility name here? and less on city?

In [ ]:
linker.m_u_parameters_chart()

In [ ]:
linker.unlinkables_chart()

### Make Predictions

In [ ]:
df_predictions = linker.predict()

In [ ]:
df_predictions.as_pandas_dataframe().sort_values(by="match_probability", ascending=False)

In [ ]:
df_preds = df_predictions.as_pandas_dataframe()

In [ ]:
df_preds[df_preds.match_probability > .5].sort_values(by="match_probability")

In [ ]:
df_preds.to_pickle("sec_eia_2005_preds")

# Take a look at predictions

Left is the SEC dataframe, right is the EIA dataframe

In [ ]:
sec_preds = df_preds.sort_values(by="match_probability", ascending=False).groupby("unique_id_l").first()

In [ ]:
eia_preds = df_preds.sort_values(by="match_probability", ascending=False).groupby("unique_id_r").first()

In [ ]:
sec_preds[sec_preds.match_probability >= .5]

In [ ]:
eia_preds[eia_preds.match_probability >= .5]

In [ ]:
def make_features(companies_sec: pd.DataFrame, utils_eia860: pd.DataFrame, block_col: str | None = "year"
    ) -> pd.DataFrame:
        """Generate comparison features based on defined features.

        The recordlinkage package helps us create feature vectors. For each column that
        we have in both datasets, this method generates a column of feature vecotrs,
        which contain values between 0 and 1 that are measures of the similarity between
        each datapoint the two datasets (1 meaning the two datapoints were exactly the
        same and 0 meaning they were not similar at all).

        For more details see recordlinkage's documentaion:
        https://recordlinkage.readthedocs.io/en/latest/ref-compare.html

        Args:
            companies_sec:
            utils_eia860:
            block_col:  If you want to restrict possible matches
                between ferc_df and eia_df based on a particular column,
                block_col is the column name of blocking column. Default is
                ``year``. If None, this method will generate features between all
                possible matches.

        Returns:
            a dataframe of feature vectors between SEC and EIA.
        """
        compare_cl = rl.Compare(
            features=[
                String(
                    "utility_name_eia",
                    "utility_name_eia",
                    label="utility_name_eia",
                    method="jarowinkler",
                ),
                String(
                    "city",
                    "city",
                    label="city",
                    method="jarowinkler",
                ),
                String(
                    "street_address",
                    "street_address",
                    label="street_address",
                    method="jarowinkler",
                ),
                String(
                    "address_2",
                    "address_2",
                    label="address_2",
                    method="jarowinkler",
                ),
                Exact(
                    "zip_code",
                    "zip_code",
                    label="zip_code",
                ),
            ]
        )

        # generate the index of all candidate features
        indexer = rl.Index()
        indexer.block(block_col)
        feature_index = indexer.index(companies_sec, utils_eia860)

        features = compare_cl.compute(feature_index, companies_sec, utils_eia860)
        return features

In [ ]:
%%time
features = make_features(companies_sec=companies_sec, utils_eia860=utils_eia860)